In [1]:
import json
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import json
import random
import time
import sys

In [3]:
# Load data from JSON file
with open("data.json", "r") as file:
    data = json.load(file)

# Initialize lists to hold our sentences (intents), labels (diseases), and responses
sentences = []
labels = []
descriptions = []
treatments = []
length = []
# Fill lists with data from JSON
for intent in data["intents"]:
    for pattern in intent["symptoms"]:
        sentences.append(pattern)
        labels.append(intent["name"])
        descriptions.append(intent["description"])
        treatments.append(intent["treatments"])
        
for x in data["intents"]:
    length.append(intent["name"])
        
print(len(length))
        
# Initialize the WordNet Lemmatizer
lemmatizer = WordNetLemmatizer()

# Stop words set
stop_words = set(stopwords.words('english'))
# Function to preprocess the text
def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

# Preprocess the sentences
sentences = [preprocess_text(sentence) for sentence in sentences]
# Create a Count vectorizer
vectorizer = CountVectorizer()
# Vectorize our sentences
X = vectorizer.fit_transform(sentences)

# Encode our labels
encoder = LabelEncoder()
y = encoder.fit_transform(labels)

# Train a model
model = MultinomialNB()
model.fit(X, y)

275


MultinomialNB()

In [5]:
def greet_user():
    greetings = ["Hi! How can I assist you today?", "Hello! How can I help you today?"]
    return random.choice(greetings)

def get_treatment(user_input):
    if "treatment" in user_input.lower() or "cure" in user_input.lower() or "treatments" in user_input.lower() or "cures" in user_input.lower() or "what are the treatments" in user_input.lower():
        return True
    else:
        return False

def possible_disease_enquiry(user_input):
    enquiry_words = ["possible disease","possible diseases","chances","chance","predicted disease","predicted illness","scope","predict"]
    return any(word in user_input.lower() for word in enquiry_words)

def is_exit_message(user_input):
    exit_words = ["thank", "thanks", "goodbye", "bye", "quit"]
    return any(word in user_input.lower() for word in exit_words)

def preprocess_text(text):
    words = nltk.word_tokenize(text)
    words = [word for word in words if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return " ".join(words)

def extract_symptoms(input_text):
    word_list = nltk.word_tokenize(input_text)
    symptoms = [lemmatizer.lemmatize(word.lower()) for word in word_list if word not in stop_words]
    return symptoms

def predict_intent(user_input):
    processed_input = preprocess_text(user_input)
    user_symptoms_vectorized = vectorizer.transform([processed_input])
    predicted_class = model.predict(user_symptoms_vectorized)
    predicted_proba = model.predict_proba(user_symptoms_vectorized)
    max_confidence = max(predicted_proba[0])
    predicted_label = encoder.inverse_transform(predicted_class)
    response_index = labels.index(predicted_label[0])
    predicted_intent = labels[response_index]
    treatment = treatments[response_index]
    description = descriptions[response_index]
    return description, predicted_intent, treatment, max_confidence

def get_possible_diseases(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []

    for intent in data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                break

    return possible_diseases

def get_possible_diseases_and_treatments(user_input):
    symptoms = extract_symptoms(user_input)
    possible_diseases = []
    treatments = {}

    for intent in data["intents"]:
        for symptom in symptoms:
            if symptom in intent["symptoms"]:
                possible_diseases.append(intent["name"])
                treatments[intent["name"]] = intent["treatments"]
                break

    return possible_diseases, treatments

def resetting_user_input(user_input):
    if "another symptoms" in user_input.lower() or "help me with another set of symptoms" in user_input.lower() or "new set of symptoms" in user_input.lower() or "fresh start" in user_input.lower() or "from beginning" in user_input.lower():
        return True
    else:
        return False

def typing_effect(message):
    for char in message:
        sys.stdout.write(char)
        sys.stdout.flush()
        time.sleep(0.05)
    print("")   

In [9]:
#Continue your code on what are the symptoms of a given disease? and what are its treatments?
typing_effect(greet_user())
symptoms_input = ""
possible_diseases = []
possible_treatments = {}

while True:
    # User enters a symptom
    user_input = input("User: ")
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue
       
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

    # Add symptom to user's symptoms
    symptoms_input += " " + user_input
    description, predicted_intent, treatment, confidence = predict_intent(symptoms_input)

    if confidence < 0.041:
        if possible_disease_enquiry(user_input):
            possible_diseases, possible_treatments = get_possible_diseases_and_treatments(symptoms_input)
            if len(possible_diseases)>0:
                typing_effect(f"Chatbot: The possible diseases based on your symptoms are: {', '.join(possible_diseases)}")
            else:
                typing_effect("No other predictions as per my knowledge")
            continue
            
        if get_treatment(user_input):
            if len(possible_diseases) > 0:
                for disease in possible_diseases:
                    typing_effect(f"Chatbot: The treatments for {disease} are: {', '.join(possible_treatments[disease])}")
            else:
                typing_effect("Chatbot: You haven't asked for possible diseases yet. Please ask for possible diseases first.")
            continue

        common_symptoms_response = ["Your symptoms are quite common, could you please provide more details or any other symptom?", 
                                   "That is a very common symptom, provide me with another symptom you are facing","That's a quite common symptom for many diseases, provide me with any another symptom you are having?"]
        sol = random.choice(common_symptoms_response)
        typing_effect(f"Chatbot: {sol}")
        typing_effect(f"Confidence: {confidence}")
        continue
        
    for item in description:
        print(*item, end="")
    print()
        
    typing_effect(f"Confidence: {confidence}")
    
    user_input = input("User: ")
    
    if get_treatment(user_input):
        typing_effect(f"Chatbot: The treatments are: {treatments}")
        continue
    
    if resetting_user_input(user_input):
        symptoms_input = ""
        possible_diseases = []
        possible_treatments = {}
        typing_effect("Sure, how can I help you")
        continue      
        
    if is_exit_message(user_input):
        typing_effect("Chatbot: You're welcome! Take care!")
        break

Hi! How can I assist you today?
User: I have nausea
Chatbot: Your symptoms are quite common, could you please provide more details or any other symptom?
Confidence: 0.018483638840785188
User: I have fever
Chatbot: That's a quite common symptom for many diseases, provide me with any another symptom you are having?
Confidence: 0.020434999887901328
User: What are the possible diseases
Chatbot: The possible diseases based on your symptoms are: Giardiasis, Gonorrhea, Group A streptococcal infection, Haemophilus influenzae infection, Hantavirus pulmonary syndrome, Helminthiasis
User: thanks
Chatbot: You're welcome! Take care!
